In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ingredient_parser
import pickle
import config
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors
import unidecode, ast
import re

In [ ]:
df = pd.read_csv(r"C:\Users\acer\OneDrive - ums.edu.my (1)\Documents\data sc\fyp\recipes.csv")
df = df.drop(['AuthorId', 'AuthorName', 'DatePublished', 'Description', 'Images', 'ReviewCount',
              'AggregatedRating', 'RecipeYield', 'Keywords', 'RecipeServings'], axis=1)

df = df.dropna() 

def transform_instructions(instructions):
    instructions_list = instructions.replace('c(', '').replace('.")', '.').replace(')', '').replace('"', '').split(',')
    instructions_list = [instruction.strip() for instruction in instructions_list]
    paragraph = ' '.join(instructions_list)
    return paragraph

def transform_ingredients(ingredients):
    ingredients_list = ingredients.replace('c(', '').replace(')', '').replace('"', '').split(',')
    ingredients_list = [ingredient.strip() for ingredient in ingredients_list]
    ingredient_string = ', '.join(ingredients_list)
    return ingredient_string

# Apply the transformation functions to the columns
df['RecipeInstructions'] = df['RecipeInstructions'].apply(transform_instructions)
df['RecipeIngredientParts'] = df['RecipeIngredientParts'].apply(transform_ingredients)

# Prompt user for input
ingredients_input = input("Enter comma separated list of ingredients: ").split(',')
prep_time_input = int(input("Enter preparation time in minutes: "))



# Convert 'prep_time' column to integer (minutes)
#df['minutes'] = df['prep_time'].astype(int)

# Function to convert PT24H20M format to minutes
def parse_duration(duration):
    hours = 0
    minutes = 0
    if 'H' in duration:
        hours = int(re.findall(r'(\d+)H', duration)[0])
    if 'M' in duration:
        minutes = int(re.findall(r'(\d+)M', duration)[0])
    return hours * 60 + minutes

df['prep_time_minutes'] = df['TotalTime'].apply(parse_duration)

# Define a transformation function to convert the format of preparation time
def transform_preparation_time(time_str):
    # Convert the string representation to a desired format
    
    # Remove the 'PT' prefix and extract the numeric portion
    numeric_str = re.sub(r'[^0-9]+', '', time_str)
    
    # Extract the hours and minutes from the numeric string
    hours = int(numeric_str[:-2]) if numeric_str[:-2] else 0
    minutes = int(numeric_str[-2:]) if numeric_str[-2:] else 0
    
    if hours == 0:
        return f"{minutes} minutes"
    else:
        return f"{hours} hours {minutes} minutes"

# Apply the transformation function to the 'TotalTime' column
df['TotalTime'] = df['TotalTime'].apply(transform_preparation_time)

# Filter dataset by user input
df = df[df['prep_time_minutes'] <= prep_time_input]


# Create a document-term matrix using the ingredients
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['RecipeIngredientParts'])
#ingredients_matrix = tfidf.fit_transform(df['ingredients'].apply(lambda x: ' '.join(x)))

# Fit the kNN model
k = 15 #Number of neighbors to consider
knn_model = NearestNeighbors(n_neighbors=k, metric='cosine')
knn_model.fit(tfidf_matrix)

# Vectorize user input ingredients
user_ingredients = tfidf.transform([','.join(ingredients_input)])

# Find k nearest neighbors
distances, indices = knn_model.kneighbors(user_ingredients)

# Initialize variables
total_overlap_percentage = 0
total_relevant_recommendations = 0
total_recommended_items = 0
total_actual_relevant_items = 0

# Define threshold for relevance
threshold = 50  

# Display recommended recipes
print("Recommended Recipes:")
for idx in indices[0]:
    recipe = df.iloc[idx]
    
    # Calculate overlap percentage
    recommended_ingredients = [ingredient.strip().lower() for ingredient in recipe['RecipeIngredientParts'].split(',')]
    recommended_ingredients_set = set(recommended_ingredients)
    user_input_lower = [ingredient.strip().lower() for ingredient in ingredients_input]
    overlap = set(user_input_lower).intersection(recommended_ingredients_set)
    overlap_percentage = len(overlap) / len(set(user_input_lower)) * 100

    # Dermine if the recommendation is relevant (overlap > threshold)
    is_relevant_recommendation = overlap_percentage >= threshold 

    total_recommended_items += 1
    total_relevant_recommendations += is_relevant_recommendation
    total_actual_relevant_items += 1  # increment if the actual item is relevant

    print("Recipe Title:", recipe['Name'])
    print("Ingredients:", recipe['RecipeIngredientParts'])
    print("Preparation Time:", recipe['TotalTime'])
    print("Instruction:", recipe['RecipeInstructions'])
    print("------------------------")

    # Print user-input and recommended ingredients
    print("User Input Ingredients:", ingredients_input)
    print("Recommended Ingredients:", recommended_ingredients)

    # Accumulate overlap percentage
    total_overlap_percentage += overlap_percentage
    
    # Print overlap%
    print("Overlap Percentage: {:.2f}%".format(overlap_percentage))
    print("------------------------")

# Calculate and print the overall accuracy-like metric
overall_accuracy = total_overlap_percentage / (k*100)
print("Overall Accuracy-like Metric: {:.2f}%".format(overall_accuracy * 100))

# Calculate precision and recall
precision = total_relevant_recommendations / total_recommended_items
recall = total_relevant_recommendations / total_actual_relevant_items

# Print precision and recall
print("Precision: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))

C:\Users\acer\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0,2,14,15,16,17,18,19,20,21,22,23,24,25) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
